SymPyBotics
===========

Symbolic Framework for Modeling and Identification of Robot Dynamics

Uses [Sympy](http://sympy.org) and [Numpy](http://www.numpy.org/) libraries.

[![Build Status](https://travis-ci.org/cdsousa/SymPyBotics.png?branch=master)](https://travis-ci.org/cdsousa/SymPyBotics)

##### Citation:
[![DOI](https://zenodo.org/badge/920/cdsousa/SymPyBotics.png)](http://dx.doi.org/10.5281/zenodo.11365)


Example
-------

Definition of a 2 DOF example robot:

In [1]:
import sympy
import sympybotics

In [2]:
rbtdef = sympybotics.RobotDef('Example Robot', # robot name
                               [('-pi/2', 0, 0, 'q+pi/2'),  # list of tuples with Denavit-Hartenberg parameters
                                ( 'pi/2', 0, 0, 'q-pi/2')], # (alpha, a, d, theta)
                               dh_convention='standard' # either 'standard' or 'modified'
                              )

In [3]:
rbtdef.frictionmodel = {'Coulomb', 'viscous'} # options are None or a combination of 'Coulomb', 'viscous' and 'offset'
rbtdef.gravityacc = sympy.Matrix([0.0, 0.0, -9.81]) # optional, this is the default value

`L` is the link inertia tensor computed about the link frame;
`l` is the link first moment of inertia;
`m` is the link mass.
These are the so-called barycentric parameters, with respect to which the dynamic model is linear.


Generation of geometric, kinematic and dynamic models:

In [4]:
rbtdef.dynparms()

[L_1xx,
 L_1xy,
 L_1xz,
 L_1yy,
 L_1yz,
 L_1zz,
 l_1x,
 l_1y,
 l_1z,
 m_1,
 fv_1,
 fc_1,
 L_2xx,
 L_2xy,
 L_2xz,
 L_2yy,
 L_2yz,
 L_2zz,
 l_2x,
 l_2y,
 l_2z,
 m_2,
 fv_2,
 fc_2]

In [5]:
rbt = sympybotics.RobotDynCode(rbtdef, verbose=True)

generating geometric model
generating forward kinematics code: transformation matrix
generating forward kinematics code: position vector
generating forward kinematics code: rotation matrix
generating kinematic model
generating jacobian code
generating dynamics model
generating inverse dynamics code
generating gravity term code
generating coriolis term code
generating coriolis matrix code
generating inertia matrix code
generating regressor matrix code
generating friction term code
done


In [6]:
rbt.geo.T[-1]

Matrix([
[-sin(q1)*sin(q2), -cos(q1),  sin(q1)*cos(q2), 0],
[ sin(q2)*cos(q1), -sin(q1), -cos(q1)*cos(q2), 0],
[         cos(q2),        0,          sin(q2), 0],
[               0,        0,                0, 1]])

In [7]:
rbt.kin.J[-1]

Matrix([
[0,        0],
[0,        0],
[0,        0],
[0, -cos(q1)],
[0, -sin(q1)],
[1,        0]])

C function generation:

In [8]:
tau_str = sympybotics.robotcodegen.robot_code_to_func('C', rbt.invdyn_code, 'tau_out', 'tau', rbtdef)

Doing print(tau_str), function code will be output:

In [9]:
print(tau_str)

void tau( double* tau_out, const double* parms, const double* q, const double* dq, const double* ddq )
{
  double x0 = -ddq[0];
  double x1 = sin(q[1]);
  double x2 = dq[0];
  double x3 = x1*x2;
  double x4 = -x3;
  double x5 = cos(q[1]);
  double x6 = -x0;
  double x7 = dq[1]*x4 + x5*x6;
  double x8 = x2*x5;
  double x9 = dq[1]*x8 + x1*x6;
  double x10 = dq[1]*parms[16] + parms[14]*x8 + parms[17]*x3;
  double x11 = 9.8100000000000005*x1;
  double x12 = dq[1]*parms[15] + parms[13]*x8 + parms[16]*x3;
  double x13 = dq[1]*parms[13] + parms[12]*x8 + parms[14]*x3;
  double x14 = 9.8100000000000005*x5;
//
  tau_out[0] = dq[0]*parms[10] + parms[11]*(((dq[0]) > 0) - ((dq[0]) < 0)) - parms[3]*x0 + x1*(ddq[1]*parms[16] - dq[1]*x13 + parms[14]*x7 + parms[17]*x9 - parms[19]*x14 + x12*x8) + x5*(ddq[1]*parms[13] + dq[1]*x10 + parms[12]*x7 + parms[14]*x9 + parms[19]*x11 + x12*x4);
  tau_out[1] = ddq[1]*parms[15] + dq[1]*parms[22] + parms[13]*x7 + parms[16]*x9 - parms[18]*x11 + parms[20]*x14 + parms[

Dynamic base parameters:

In [10]:
rbt.calc_base_parms(verbose=True)
rbt.dyn.baseparms

calculating base parameters and regressor code
done


Matrix([
[L_1yy + L_2zz],
[         fv_1],
[         fc_1],
[L_2xx - L_2zz],
[        L_2xy],
[        L_2xz],
[        L_2yy],
[        L_2yz],
[         l_2x],
[         l_2z],
[         fv_2],
[         fc_2]])

Author
------

[Cristóvão Duarte Sousa](https://github.com/cdsousa)

Install
-------

From git source:

    git clone https://github.com/cdsousa/SymPyBotics.git
    cd sympybotics
    python setup.py install

License
-------

New BSD license. See [License File](LICENSE.txt)
